### Задача 3

*Эту задачу можно выполнить в Питоне. Можно в отдельном ноутбуке.*

Проведите эксперимент по определению реального уровня значимости критерия для проверки гипотезы о незначимости коэффициента в гауссовской линейной модели, если на самом деле в данных присутствует гетероскедастичность. 
Для этого смоделируйте некоторым образом двумерные данные $x$ и посчитайте по ним ожидаемый отклик 
	$y(x) = \theta_0 + \theta_1 x^{(1)} + \theta_2 x^{(2)}$, где коэффициенты выберите по своему усмотрению, причем $\theta_2 = 0$. 
Зашумите набор значений $y(x_i)$ некоторым шумом, дисперсия которого зависит от $x$ или от номера наблюдения. 
По таким данным обучите	линейную модель и проверьте гипотезу $\mathsf{H}_0\colon \theta_2 = 0$. 
Повторите эксперимент несколько раз и посчитайте долю случаев, в которых гипотеза отвергается. Распределение шума должно быть одинаковым в каждом эксперименте.

In [98]:
import scipy.stats as sps
import numpy as np

# Смоделируем данные
n = 100
X = sps.multivariate_normal(mean=[0, 0], cov=[[100, 0], [0, 100]]).rvs(size=n)
theta_0 = 10
theta_1 = 1
theta_2 = 0
X = np.hstack((np.array([1] * n).reshape(n, -1), X))
y_true = X @ np.array([theta_0, theta_1, theta_2])

# Зашумим истинные значения
y = y_true.reshape(n, -1) +\
sps.norm(scale = np.abs(X[:, 1].reshape(n, -1) / 5)).rvs(size=(n, 1))

In [99]:
import statsmodels.api as sm

# Обучим ЛР и посмотрим на гипотезы о незначимости
model = sm.OLS(y, X)
res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.966
Method:                 Least Squares   F-statistic:                     1397.
Date:                Tue, 16 Feb 2021   Prob (F-statistic):           3.20e-72
Time:                        12:37:39   Log-Likelihood:                -205.51
No. Observations:                 100   AIC:                             417.0
Df Residuals:                      97   BIC:                             424.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.8363      0.192     51.192      0.000       9.455      10.218
x1             0.9521      0.018     52.849      0.000       0.916       0.988
x2            -0.0328      0.022     -1.498      0.137      -0.076       0.011
==============================================================================
Omnibus:                       41.282   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              230.230
Skew:                          -1.137   Prob(JB):                     1.01e-50
Kurtosis:                      10.077   Cond. No.                         10.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Вывод.** Как мы видим, p-value для x2 равен 0.137, то есть гипотеза о незначимости коэффициента не отвергается.

In [110]:
# Проведём этот эксперимент несколько раз и найдём долю отвержений

tests_cnt = 100
for test_num in range(10):
    rejected_cnt = 0
    for i in range(tests_cnt):
        X = sps.multivariate_normal(
            mean=[0, 0], cov=[[100, 0], [0, 100]]).rvs(size=n)
        X = np.hstack((np.array([1] * n).reshape(n, -1), X))
        y_true = X @ np.array([theta_0, theta_1, theta_2])

        # Хороший вопрос, что имелось в виду в условии под "одинаковым распределением шума"
        # вставлять один и тот же шум нельзя: потеряется зависимость от текущих х
        # так что параметры распределения шума будут разными в разных экспериментах
        y = y_true.reshape(n, -1) +\
        sps.norm(scale = np.abs(X[:, 1].reshape(n, -1) / 5)).rvs(size=(n, 1))

        model = sm.OLS(y, X)
        res = model.fit()
        res.summary()

        pval = float(res.summary().tables[1].data[3][4])
        if pval <= 0.05:
            rejected_cnt += 1

    print("Доля случаев отвержения гипотез: {0:.3f}".format(rejected_cnt / tests_cnt))

Доля случаев отвержения гипотез: 0.020
Доля случаев отвержения гипотез: 0.040
Доля случаев отвержения гипотез: 0.040
Доля случаев отвержения гипотез: 0.060
Доля случаев отвержения гипотез: 0.040
Доля случаев отвержения гипотез: 0.020
Доля случаев отвержения гипотез: 0.050
Доля случаев отвержения гипотез: 0.040
Доля случаев отвержения гипотез: 0.040
Доля случаев отвержения гипотез: 0.040


**Вывод.** Таким образом, всё равно есть небольшая доля случаев, в которых происходит отвержение $H_0$, но заметим, что доля таких случаев часто не превышает 0.05. Похоже, что это связано с тем, что p <= 0.05 (то есть веротяность ошибки первого рода соответствующего критерия <= 0.05).